# The problem: What are the total minutes streamed per platform of the users in a country

In [ ]:
import numpy

Rule of Thumb: 2/3 Rule

Country → Minutes (8 neurons)

- Simpler problem: Predicting single number (total minutes)
- Less complexity: Regression typically needs fewer neurons than classification
- Reasoning: Total streaming time has fewer patterns than genre preferences